In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda

from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

import wandb

In [5]:
dir_list = sorted(os.listdir("./CUB_200_2011/images/"))
x_train, x_test = [],[]
y_train, y_test = [],[]

for i in dir_list:
    cnt = 0 
    for j in os.listdir("./CUB_200_2011/images/"+i):
        img = plt.imread("./CUB_200_2011/images/"+i+"/"+j)
        y = int(i.split(".")[0])
        
        if cnt<9:
            x_test.append(img)
            y_test.append(y)
            cnt += 1
        else:
            x_train.append(img)
            y_train.append(y)

In [6]:
def preprocessing(image):
    resized_image = tf.image.resize(image, [112,112])
    return tf.keras.applications.vgg16.preprocess_input(resized_image)

X_train = []

for _ in x_train:
    if len(img.shape) == 3 or len(img.shape) == 4:
        img = preprocessing(img)
        X_train.append(img)
        
X_train = np.array(X_train)
y_train = np.array(y_train)
y_train = tf.keras.utils.to_categorical(y_train)



X_test = []

for _ in x_test:
    if len(img.shape) == 3 or len(img.shape) == 4:
        img = preprocessing(img)
        X_test.append(img)
        
X_test = np.array(X_test)
y_test = np.array(y_test)       
y_test = tf.keras.utils.to_categorical (y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

2023-02-14 07:57:02.615386: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 147.0KiB (rounded to 150528)requested by op BiasAdd
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-02-14 07:57:02.615519: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-02-14 07:57:02.615552: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 2, Chunks in use: 2. 512B allocated for chunks. 512B in use in bin. 24B client-requested in use in bin.
2023-02-14 07:57:02.615567: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-02-14 07:57:02.615584: I tensorflow/core/common_runtime/

ResourceExhaustedError: {{function_node __wrapped__BiasAdd_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[112,112,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:BiasAdd]

In [ ]:
from sklearn.model_selection import train_test_split

X_tn, X_val, y_tn, y_val = train_test_split(X_train, y_train, test_size=0.18, random_state=42)

In [ ]:
print(X_tn.shape) 
print(y_tn.shape) 
print(X_test.shape)
print(y_test.shape)
print(X_val.shape) 
print(y_val.shape)

In [ ]:
wandb.init(project="Bird_efficientNet", entity="hcim", name='denseNet')

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D,AveragePooling2D, GlobalAveragePooling2D, Flatten, Dense, Activation, MaxPool2D, BatchNormalization, Dropout,RandomFlip,RandomRotation

densenet = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape=(112,112,3))

x = densenet.output
x = Flatten()(densenet.output)
predictions = Dense(201, activation='softmax')(x)

model = Model(inputs=densenet.input, outputs=predictions)
model.summary()

In [ ]:
from tensorflow.keras import callbacks
class WandbCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        wandb.log(logs)
        
        

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=15, horizontal_flip=True, vertical_flip=True, brightness_range=[0.01, 0.1], channel_shift_range=10)
datagen.fit(X_tn)

In [ ]:
optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=10e-4)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.CategoricalAccuracy()]) 

hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=64), epochs=150, validation_data=(X_val, y_val), callbacks=[WandbCallback()])

In [ ]:
print(model.evaluate(X_train, y_train))
print(model.evaluate(X_val, y_val))
print(model.evaluate(X_test, y_test))